In [62]:
import glob 
from pandas import read_csv, concat, DataFrame

#diretório
input_dir = r"C:\Users\Natan\Documents\TPAD2\dados"

##### **Listando os arquivos** #####

In [63]:
arquivos_ies = glob.glob(r"C:\Users\Natan\Documents\TPAD2\dados/*IES*.CSV")
arquivos_cursos = glob.glob(r"C:\Users\Natan\Documents\TPAD2\dados/*Cadastro_Cursos*.CSV")

##### **Filtrando colunas** #####

In [64]:
colunas_ies = ['CO_IES','SG_IES','TP_CATEGORIA_ADMINISTRATIVA','TP_ORGANIZACAO_ACADEMICA']

#Diurno
colunas_cadastro = ['NU_ANO_CENSO','SG_UF','CO_IES','QT_ING_DIURNO','QT_CONC_DIURNO','TP_ORGANIZACAO_ACADEMICA','TP_CATEGORIA_ADMINISTRATIVA','NO_CINE_ROTULO']
#Noturno
colunas_cadastro_noturno = ['NU_ANO_CENSO','SG_UF','CO_IES','QT_ING_NOTURNO','QT_CONC_NOTURNO','TP_ORGANIZACAO_ACADEMICA','TP_CATEGORIA_ADMINISTRATIVA','NO_CINE_ROTULO']

##### **Leitura dos dados e Correção de tipagem** #####

In [65]:
#Arquivos da cadastro cursos - Diurno
tabela_cadastro = []

for arquivo in arquivos_cursos:
    tabela_cadastro.append(read_csv(arquivo, sep=';', encoding='ISO 8859-1',usecols=colunas_cadastro)
    .rename(columns = {'NU_ANO_CENSO':'ano'})
    .query('TP_ORGANIZACAO_ACADEMICA==1 & TP_CATEGORIA_ADMINISTRATIVA==1')
    .groupby(['ano', 'CO_IES','NO_CINE_ROTULO']).agg(
        ingressantes_diurno = ('QT_ING_DIURNO', 'sum'),
        concluintes_diurno =('QT_CONC_DIURNO', 'sum')
        #nome_curso=('NO_CINE_ROTULO','max'))
    ))

cdd = concat(tabela_cadastro).reset_index()

#Convertendo p/ inteiros
cdd['ingressantes_diurno'] = cdd['ingressantes_diurno'].astype(int)
cdd['concluintes_diurno'] = cdd['concluintes_diurno'].astype(int)

#Arquivos da cadastro ies 
tabela_ies = []

for arquivo in arquivos_ies:
    tabela_ies.append(read_csv(arquivo, sep=';', encoding='ISO 8859-1',usecols=colunas_ies)
    .query('TP_ORGANIZACAO_ACADEMICA==1 & TP_CATEGORIA_ADMINISTRATIVA==1')
    )

ies = concat(tabela_ies)

####################################################################

#Arquivos da cadastro cursos - Noturno
tabela_cadastro_noturno = []

for arquivo2 in arquivos_cursos:
    tabela_cadastro_noturno.append(read_csv(arquivo2, sep=';', encoding='ISO 8859-1',usecols=colunas_cadastro_noturno)
    .rename(columns = {'NU_ANO_CENSO':'ano'})
    .query('TP_ORGANIZACAO_ACADEMICA==1 & TP_CATEGORIA_ADMINISTRATIVA==1')
    .groupby(['ano', 'CO_IES','NO_CINE_ROTULO']).agg(ingressantes_noturno = ('QT_ING_NOTURNO', 'sum'), concluintes_noturno =('QT_CONC_NOTURNO', 'sum'))
    )

cdn = concat(tabela_cadastro_noturno).reset_index()

#Convertendo p/ inteiros
cdn['ingressantes_noturno'] = cdn['ingressantes_noturno'].astype(int)
cdn['concluintes_noturno'] = cdn['concluintes_noturno'].astype(int)

##### **Agrupando os dados por curso e ano, nessa ordem (apenas UFPB)** #####

In [66]:
####################################################################

#Evasão - Diurno *dfd = dataframe diurno
dfd = ies.merge(cdd, on='CO_IES', how='inner')
dfd = dfd.loc[dfd['SG_IES']=='UFPB']

dfd = (dfd
.groupby(['NO_CINE_ROTULO','ano']).agg(
    ingressantes=('ingressantes_diurno','sum'),
    concluintes=('concluintes_diurno','sum')
).reset_index())

####################################################################

#Evasão - Noturno *dfn = dataframe noturno
dfn = ies.merge(cdn, on='CO_IES', how='inner')
dfn = dfn.loc[dfn['SG_IES']=='UFPB']

dfn = (dfn
.groupby(['NO_CINE_ROTULO','ano']).agg(
    ingressantes=('ingressantes_noturno','sum'),
    concluintes=('concluintes_noturno','sum')
).reset_index())

##### **Cursos CCSA (P/ fins de consulta)** #####

In [67]:

'Administração'
'Economia'
'Arquivologia'     #só noturno
'Biblioteconomia'  #só noturno
'Ciências atuariais'
'Contabilidade'
'Gestão pública'
'Relações internacionais'

'Relações internacionais'

##### **Função p/ consultar cursos** #####

In [120]:
def consulta_curso():
    curso = str(input('Digite o nome do curso: '))
    turno = str(input('Digite o turno: '))
    #df = DataFrame
    #df2 = DataFrame
    
    if turno == 'diurno':
        df = DataFrame
        df = (dfd.loc[dfd['NO_CINE_ROTULO']==curso]
        .groupby(['NO_CINE_ROTULO','ano']).agg(
        ingressantes=('ingressantes','sum'),
        concluintes=('concluintes','sum')
        ).assign(evasão = lambda x: (1-(x.concluintes/x.ingressantes.shift(5)))*100)
        .reset_index())
        df['turno'] = 'diurno'
        return df.loc[df['ano']>2014]
        
    elif turno == 'noturno':
        df = DataFrame
        df = (dfn.loc[dfn['NO_CINE_ROTULO']==curso]
        .groupby(['NO_CINE_ROTULO','ano']).agg(
        ingressantes=('ingressantes','sum'),
        concluintes=('concluintes','sum')
        ).assign(evasão = lambda x: (1-(x.concluintes/x.ingressantes.shift(5)))*100)
        .reset_index())
        df['turno'] = 'noturno'
        return df.loc[df['ano']>2014]

    else:
        df = (dfd.loc[dfd['NO_CINE_ROTULO']==curso]
        .groupby(['NO_CINE_ROTULO','ano']).agg(
        ingressantes=('ingressantes','sum'),
        concluintes=('concluintes','sum')
        ).assign(evasão = lambda x: (1-(x.concluintes/x.ingressantes.shift(5)))*100)
        .reset_index())
        df['turno'] = 'diurno'
    
        df2 = (dfn.loc[dfn['NO_CINE_ROTULO']==curso]
        .groupby(['NO_CINE_ROTULO','ano']).agg(
        ingressantes=('ingressantes','sum'),
        concluintes=('concluintes','sum')
        ).assign(evasão = lambda x: (1-(x.concluintes/x.ingressantes.shift(5)))*100)
        .reset_index())
        df2['turno'] = 'noturno'
        df3 = concat([df,df2])
        return df3.dropna()

In [129]:
consulta_curso()

,NO_CINE_ROTULO,ano,ingressantes,concluintes,evasão,turno
5,Gestão pública,2015,336,204,66.666667,diurno
6,Gestão pública,2016,588,324,47.058824,diurno
7,Gestão pública,2017,1056,180,70.000000,diurno
8,Gestão pública,2018,1020,132,77.551020,diurno
9,Gestão pública,2019,972,84,81.578947,diurno
10,Gestão pública,2020,1212,12,96.428571,diurno
5,Gestão pública,2015,528,360,40.000000,noturno
6,Gestão pública,2016,348,276,54.000000,noturno
7,Gestão pública,2017,0,168,70.833333,noturno
8,Gestão pública,2018,0,120,82.142857,noturno


In [122]:
econo = consulta_curso().to_excel('economia.xlsx')